In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import time

In [4]:
filenames = [
             "202012-baywheels-tripdata",]

count = 0
for filename in filenames:

    bike = pd.read_csv(filename)
    count += len(bike)
    
print(count)

806723


In [7]:
bike = pd.read_csv("bike.csv")
bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   duration_sec             19 non-null     int64  
 1   start_time               19 non-null     object 
 2   end_time                 19 non-null     object 
 3   start_station_id         19 non-null     int64  
 4   start_station_name       19 non-null     object 
 5   start_station_latitude   19 non-null     float64
 6   start_station_longitude  19 non-null     float64
 7   end_station_id           19 non-null     int64  
 8   end_station_name         19 non-null     object 
 9   end_station_latitude     19 non-null     float64
 10  end_station_longitude    19 non-null     float64
 11  bike_id                  19 non-null     int64  
 12  user_type                19 non-null     object 
dtypes: float64(4), int64(4), object(5)
memory usage: 2.1+ KB


,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type
0,80110,12/31/17 16:57,1/1/18 15:12,74,Laguna St at Hayes St,37.776435,-122.426244,43,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96,Customer
1,78800,12/31/17 15:56,1/1/18 13:49,284,Yerba Buena Center for the Arts (Howard St at ...,37.784872,-122.400876,96,Dolores St at 15th St,37.766210,-122.426614,88,Customer
2,45768,12/31/17 22:45,1/1/18 11:28,245,Downtown Berkeley BART,37.870348,-122.267764,245,Downtown Berkeley BART,37.870348,-122.267764,1094,Customer
3,62172,12/31/17 17:31,1/1/18 10:47,60,8th St at Ringold St,37.774520,-122.409449,5,Powell St BART Station (Market St at 5th St),37.783899,-122.408445,2831,Customer
4,43603,12/31/17 14:23,1/1/18 2:29,239,Bancroft Way at Telegraph Ave,37.868813,-122.258764,247,Fulton St at Bancroft Way,37.867789,-122.265896,3167,Subscriber


In [8]:
type(bike_poc["start_time"][1])

str

In [9]:
bike_poc["start_time"][0].replace(' ', '_').split(':', 1)[0]

'12/31/17_16'

In [10]:
bike_poc["start_time"] = [time.replace(' ', '_').split(':', 1)[0] for time in bike_poc["start_time"]]
bike_poc["end_time"] = [time.replace(' ', '_').split(':', 1)[0] for time in bike_poc["end_time"]]
bike_poc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   duration_sec             19 non-null     int64  
 1   start_time               19 non-null     object 
 2   end_time                 19 non-null     object 
 3   start_station_id         19 non-null     int64  
 4   start_station_name       19 non-null     object 
 5   start_station_latitude   19 non-null     float64
 6   start_station_longitude  19 non-null     float64
 7   end_station_id           19 non-null     int64  
 8   end_station_name         19 non-null     object 
 9   end_station_latitude     19 non-null     float64
 10  end_station_longitude    19 non-null     float64
 11  bike_id                  19 non-null     int64  
 12  user_type                19 non-null     object 
dtypes: float64(4), int64(4), object(5)
memory usage: 2.1+ KB


In [11]:
station_zipcode = defaultdict(str)
station_zipcode[250] = 94702
station_zipcode[309] = 95112

In [12]:
missing_zipcode = []

In [13]:
import geopy

def get_zipcode(station_num, lat, lon):
    zipcode = station_zipcode.get(station_num, 0)
    if not zipcode:
        try:
            zipcode = geolocator.reverse((lat, lon)).raw['address']['postcode'][:5]
            station_zipcode[station_num] = zipcode
        except:
            missing_zipcode.append((station_num, lat, lon))
            print("Having trouble retriving postcode for station:", station_num, "with lat:", lat, " and lon:", lon)
        
    return zipcode

geolocator = geopy.Nominatim(user_agent='cyson')

# get_zipcode(74, '37.776435', '-122.426244')

In [14]:
def process_bike_data(file, df):
    station_flow = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    
    # format datetime
    file["start_time"] = [time.replace(' ', '_').split(':', 1)[0] for time in file["start_time"]]
    file["end_time"] = [time.replace(' ', '_').split(':', 1)[0] for time in file["end_time"]]

    for index, row in file.iterrows():
        start_datetime = row['start_time']
        end_datetime = row['end_time']
        start_zipcode = get_zipcode(row['start_station_id'], \
                                    row['start_station_latitude'], \
                                    row['start_station_longitude'])
        end_zipcode = get_zipcode(row['end_station_id'], \
                                    row['end_station_latitude'], \
                                    row['end_station_longitude'])
        station_flow[start_zipcode][start_datetime]['rental'] += 1
        station_flow[start_zipcode][start_datetime]['change'] -= 1
        station_flow[end_zipcode][end_datetime]['change'] += 1

    for zipcode, v in station_flow.items():
        for datetime, item in v.items():
            df = df.append({'zipcode': zipcode, 'datetime': datetime, \
                            'change': item['change'], 'rental': item['rental']}, \
                           ignore_index=True)
            
    return df

In [18]:
# filenames = ["201801-fordgobike-tripdata.csv", "201802-fordgobike-tripdata.csv",\
#              "201803-fordgobike-tripdata.csv", "201804-fordgobike-tripdata.csv",\
#              "201805-fordgobike-tripdata.csv", "201806-fordgobike-tripdata.csv",\
#              "201807-fordgobike-tripdata.csv", "201808-fordgobike-tripdata.csv",\
#              "201809-fordgobike-tripdata.csv", "201810-fordgobike-tripdata.csv",\
#              "201811-fordgobike-tripdata.csv", "201812-fordgobike-tripdata.csv",\
#              "201901-fordgobike-tripdata.csv", "201902-fordgobike-tripdata.csv",\
#              "201903-fordgobike-tripdata.csv", "201904-fordgobike-tripdata.csv",\
#              "201905-fordgobike-tripdata.csv", "201906-fordgobike-tripdata.csv"]

filenames = ["201905-baywheels-tripdata.csv", "201906-baywheels-tripdata.csv"]

for filename in filenames:
    tic = time.time()

    bike_raw_data = pd.read_csv(filename)
    df = pd.DataFrame(columns=['zipcode', 'datetime', 'change', 'rental'])
    df = process_bike_data(bike_raw_data, df)

    df.to_csv("lyft_data.csv", mode='a', header=False)

    toc = time.time()
    
    print('time:', round(toc-tic, 2),'s')
    

time: 110.14 s


/Users/xingchen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Having trouble retriving postcode for station: nan with lat: 37.3373331667  and lon: -121.8869441667
Having trouble retriving postcode for station: nan with lat: 37.3383633333  and lon: -121.8854398333
Having trouble retriving postcode for station: nan with lat: 37.3376425  and lon: -121.8856496667
Having trouble retriving postcode for station: nan with lat: 37.3378933333  and lon: -121.8858576667
Having trouble retriving postcode for station: nan with lat: 37.3375988333  and lon: -121.8855306667
Having trouble retriving postcode for station: nan with lat: 37.3376621667  and lon: -121.8856216667
Having trouble retriving postcode for station: nan with lat: 37.3376938333  and lon: -121.88546550000001
Having trouble retriving postcode for station: nan with lat: 37.3377376667  and lon: -121.8853088333
Having trouble retriving postcode for station: nan with lat: 37.337676  and lon: -121.8853395
Having trouble retriving postcode for station: nan with lat: 37.3384216667  and lon: -121.8857116

In [22]:
lyft_data = pd.read_csv("lyft_data.csv")
lyft_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452571 entries, 0 to 452570
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   0              452571 non-null  int64 
 1   94114          452571 non-null  object
 2   2018-01-31_22  452571 non-null  object
 3   -1             452571 non-null  int64 
 4   4              452571 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 17.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 452571 entries, 0 to 452570
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   0              452571 non-null  int64 
 1   94114          452571 non-null  object
 2   2018-01-31_22  452571 non-null  object
 3   -1             452571 non-null  int64 
 4   4              452571 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 20.7+ MB


/Users/xingchen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
df.head()

,zipcode,datetime,change
0,94612,2018-12-31_20,2
1,94612,2019-01-01_15,1
2,94612,2018-12-31_23,0
3,94612,2018-12-31_22,-2
4,94612,2018-12-31_21,-1


In [29]:
muni_data = pd.read_csv("muni_data_clean.csv")
weather_data = pd.read_csv("weather_data.csv")
parking_data = pd.read_csv("park_data_cleaned.csv")


,Unnamed: 0,STOPNAME,STOPID,LATITUDE,LONGITUDE,ATSTREET,Neighborhoods,ORIENTATION,POSITION
0,0,Palou Ave&Hawes St E-NS/PS,5866,37.728679,-122.381425,HAWES ST,78.0,E,NS
1,1,California St&Leavenworth St NE-NS,3877,37.791285,-122.415611,LEAVENWORTH ST,16.0,NE,NS
2,2,Carmel St&Belvedere St SW-MI/SB,4204,37.760920,-122.447631,BELVEDERE ST,114.0,SW,MI
3,3,Beach St&Mason St SW-NS/PS,3092,37.807407,-122.414125,MASON ST,99.0,SW,NS
4,4,Paul Ave&San Bruno Ave N-MB/SB,5904,37.723879,-122.401576,SAN BRUNO AVE,91.0,N,MB


In [53]:
muni_zipcode = []
 
for idx, muni in muni_data.iterrows():
    lat = muni["LATITUDE"]
    lon = muni["LONGITUDE"]
    try:
        zipcode = geolocator.reverse((lat, lon)).raw['address']['postcode'][:5]
        muni_zipcode.append(zipcode)
    except:
        print("Having trouble retriving postcode for station:", "with lat:", lat, " and lon:", lon)
        

muni_data['zipcode'] = muni_zipcode

Having trouble retriving postcode for station: with lat: 37.83238  and lon: -122.53867


ValueError: Length of values (3489) does not match length of index (3490)

In [57]:
muni_data['zipcode'] = muni_zipcode

In [60]:
muni_data.to_csv("muni_data_cleaned.csv")

In [ ]:
# clean weather data again
weather_data